** **bold text**Task 1. Generating training dataset** 

Since logistic regression is a classification problem with two classes, we need a labelled dataset with two classes as the training set. Complete the function to obtain the datapoints corresponding to labels *'ship'* and *'car'*. 

steps to build a new dataset with these coditions: 

1. Loading the train and test sets of CIFAR 10 from torchvision using a batch size of 1024.
2. Splitting the training samples by 80:20 ratio into train set and validation set respectively. 
3. Filtering the datasets to only have images with classes 'ship' or 'car'. The corresponding labels are 8 and 1 respectively.
4. The new labels for binary classification problem should be 'ship' : 0 and 'car': 1.
5. Define a [torch.utils.data.Dataset](https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html) with the filtered tensors of images and the newly created labels.
6. Define a dataloader for training and validation datasets with batch_size 64.



In [0]:
# TODO : generate the train, validation, and test sets from CIFAR 10  
import os
import torch, torchvision
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import utils, datasets
from torch import nn,autograd
from torch.autograd import Variable
import warnings
from collections import Counter
from torch import FloatTensor
from torch import optim
from torchvision import transforms, utils, datasets

import torch.nn.functional as F
use_cuda = torch.cuda.is_available()
warnings.filterwarnings("ignore")



In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #mean, std for Red, Green and Blue
])
train = datasets.CIFAR10(root='data/cifar/', train=True, transform=transform, target_transform=None, download=True)
test =datasets.CIFAR10(root='data/cifar/', train=False, transform=transform, target_transform=None, download=True)

0it [00:00, ?it/s]

 98%|█████████▊| 167387136/170498071 [00:11<00:00, 34861345.56it/s]

Extracting data/cifar/cifar-10-python.tar.gz to data/cifar/
Files already downloaded and verified


In [0]:

class TrainCIFARDataset(Dataset):
    

    def __init__(self, file):
        self.file = self.filter_dataset(file)
    
 
    def __len__(self):
        return len(self.file)
    def __getitem__(self, idx):
        return self.file[idx]

    def replace_target_values(self ,t):
        t = list(t)
        targets = {8:0, 1:1}
        t[1] = targets[t[1]]
        return tuple(t)

    def filter_dataset(self, data):
        """
        filter the data to get only classes with ship and car, then replace the values of ships and car to 0,1
        """
        targets = {8:0, 1:1}
        filtered_ = [data[i] for i in range(len(data)) if data[i][1] in targets] 
        filtered_ = list(map(self.replace_target_values, filtered_)) 
        return filtered_

class TestCIFARDataset(TrainCIFARDataset):
    def __init__(self, file):
        self.file = self.filter_dataset(file)
    def __len__(self):
        return len(self.file)

    def __getitem__(self, idx):
        return self.file[idx]


In [14]:
train_data = TrainCIFARDataset(train)
train_loader = DataLoader(train_data, batch_size=1024)
test_dataset = TestCIFARDataset(test)
test_dataset = DataLoader(test_dataset, batch_size=1024)

train_size = int(0.8 * len(train_data))
test_size = len(train_data) - train_size


train_dataset, val_dataset = random_split(train_data, [train_size, test_size])
train_dataset, val_dataset = DataLoader(train_dataset, batch_size=64), DataLoader(val_dataset, batch_size= 64)

170500096it [00:30, 34861345.56it/s]                               

In [27]:
from tqdm.autonotebook import tqdm

model = nn.Sequential(nn.Linear(3072, 1000),
                      nn.ReLU(),
                      nn.Dropout2d(),
                      nn.Linear(1000, 600),
                      nn.ReLU(),
                      nn.Linear(600, 200),
                      nn.ReLU(),
                      nn.Dropout2d(),
                      nn.Linear(200, 50),
                      nn.ReLU(),
                      nn.Linear(50, 2),
                      nn.Softmax())


model.to(device ='cuda')
criterion = nn.CrossEntropyLoss()
learning_rate = .001
reg = 0.02
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg)
# print(model, X_pt.shape, Y_pt.shape,X_pt)
num_epochs = 40
total_loss = 0.0
for _ in tqdm(range(num_epochs)):
    for X, Y in train_dataset:
        X=X.view(64,3072)
        optimizer.zero_grad()
        predictions = model(X.to(device='cuda'))
        loss_this_epoch = criterion(predictions, Y.to(device='cuda'))
        loss_this_epoch.backward()
        optimizer.step()
        total_loss += loss_this_epoch.item()

    # print(total_loss)
    total_loss = 0.0




In [30]:
correct = 0
total = 0
use_cuda = torch.cuda.is_available()
device = torch.device('cpu')

model.eval()
with torch.no_grad():
    for data in tqdm(test_dataset):
        images, labels = data
        images = images.to(device='cuda') 
        labels = labels.to(device='cuda')
        images = images.view(images.shape[0],3072)
        outputs = model(images)
        predicted = torch.argmax(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


Accuracy of the network on the 10000 test images: 80 %
